## dbt Cloud API Example for Getting Database Credentials in AWS Secrets Manager
- This can be done via scripting with the dbt Cloud APIs or VIA the [dbt Cloud Terraform Provider](https://registry.terraform.io/providers/dbt-labs/dbtcloud/latest)

### The following dbt Cloud APIs are used during this script
- [List dbt Cloud Credential Metadata API Call](https://docs.getdbt.com/dbt-cloud/api-v3#/operations/List%20Credentials)
- [Partial Update of dbt Cloud Credentials API Call](https://docs.getdbt.com/dbt-cloud/api-v3#/operations/Partial%20Update%20Credentials)


___

In [31]:
import boto3
import json
import requests
from botocore.exceptions import ClientError
import os

___

#### __Step 1:__ Function to get secrets from aws secrets manager

In [4]:
def get_secret(secret_name, region_name):
    """
    Retrieve a secret's value from AWS Secrets Manager.

    :param secret_name: The name or ARN of the secret in Secrets Manager.
    :param region_name: The AWS region where the secret is stored.
    :return: The secret value as a string (or dictionary if it's JSON).
    """
    # Create a Secrets Manager client
    client = boto3.client('secretsmanager', region_name=region_name)

    try:
        # Fetch the secret value
        get_secret_value_response = client.get_secret_value(SecretId=secret_name)
    except ClientError as e:
        print(f"Error retrieving secret {secret_name}: {e}")
        return None
    else:
        # Check if secret is string or binary
        if 'SecretString' in get_secret_value_response:
            secret_string = get_secret_value_response['SecretString']
            # If it's JSON, you can load it into a dictionary
            try:
                secret_dict = json.loads(secret_string)
                return secret_dict
            except json.JSONDecodeError:
                # Not JSON, return the raw string
                return secret_string
        else:
            # Secret is stored as bytes
            secret_binary = get_secret_value_response['SecretBinary']
            return secret_binary

#### __Step 2:__ ___(OPTIONAL)___ Function to rotate a key in AWS Secrets manager
- Note: This requires some additional set up in AWS secrets manager such as a lambda function to rotate the keys

In [8]:
def rotate_secret(secret_id, region_name):
    """
    Triggers an immediate rotation of a secret that already has rotation enabled.

    :param secret_id: The name or ARN of the secret.
    :param region_name: The AWS region (e.g., "us-east-1").
    :return: Response from Secrets Manager's rotate_secret call.
    """
    client = boto3.client('secretsmanager', region_name=region_name)

    try:
        response = client.rotate_secret(
            SecretId=secret_id,
            RotateImmediately=True  # Rotate now instead of waiting for the next scheduled rotation
        )
        print("Rotation triggered successfully!")
        return response
    except ClientError as e:
        print(f"Error rotating secret {secret_id}: {e}")
        raise

#### __Step 3:__ Function to list the credential ids of all credentials in dbt Cloud
- Note: You can find your dbt Cloud Access URL under Account Settings >> Access URL (Provide it in this format: `cloud.getdbt.com` or `ez706.us1.dbt.com`)

In [63]:
def list_dbt_cloud_credentials(account_id: int, project_id: int, dbt_cloud_token: str, dbt_cloud_access_url='cloud.getdbt.com') -> dict:
    """
    Retrieve a list of credentials for a specific dbt Cloud account and project.

    :param account_id: The dbt Cloud account ID.
    :param project_id: The dbt Cloud project ID.
    :param dbt_cloud_token: The API token for authentication (format: 'Token <TOKEN>').
    :return: A dictionary containing the response from the dbt Cloud API.
    :raises requests.exceptions.RequestException: If the request fails.
    """

    url = (
        f"https://{dbt_cloud_access_url}/api/v3/"
        f"accounts/{account_id}/projects/{project_id}/credentials/"
    )

    # dbt Cloud requires the Authorization header
    headers = {
        "Content-Type": "application/json",
        "Authorization": f"Token {dbt_cloud_token}"
    }

    try:
        # Make the GET request
        response = requests.get(url, headers=headers)

        # Raise an HTTPError for bad responses (4xx or 5xx)
        response.raise_for_status()

        # Parse and return the JSON response
        response_data = response.json()['data']
    except requests.exceptions.RequestException as e:
        print(f"Failed to retrieve credentials. Error: {e}")
        raise

    extracted_data = []
    for item in response_data:
        extracted_data.append({
            "credential_id": item.get("id"),
            "method": item.get("method"),
            "username": item.get("username"),
            "default_schema": item.get("default_schema")
        })
    return extracted_data

#### __Step 4:__ Function to grab the credential metadata from dbt Cloud and update it with the new secret

In [64]:
def update_dbt_cloud_database_connection_credentials(
    account_id: int,
    project_id: int,
    credential_id: int,
    dbt_cloud_token: str,
    username: str,
    password: str,
    dbt_cloud_access_url='cloud.getdbt.com'):

    url = (
        f"https://{dbt_cloud_access_url}/api/v3/"
        f"accounts/{account_id}/projects/{project_id}/credentials/{credential_id}"
    )
    
    # dbt Cloud uses the "Authorization: Token <YOUR_TOKEN>" header
    headers = {
        "Content-Type": "application/json",
        "Authorization": f"Token {dbt_cloud_token}"
    }

    body_payload = json.dumps({"username":username, "password": password})
    
    # Make the PATCH request
    response = requests.patch(url, headers=headers, data=body_payload)
    
    # Raise an HTTPError if the status code indicates an error (4xx or 5xx)
    response.raise_for_status()
    
    return response.json()

#### __Step 5:__ Putting it all together -- Listing cred ids, getting the new secret from AWS Secrets manager and updating dbt Cloud

___Listing all credential metadata___

In [65]:
dbt_cloud_api_token = os.getenv("DBT_CLOUD_API_TOKEN") 
# can set this locally via export DBT_CLOUD_API_TOKEN="<<api service account token>>"
dbt_cloud_account_id = 123456
dbt_cloud_project_id = 654321

# list all credentials for the dbt Cloud account and project
dbt_cloud_credential_ids = list_dbt_cloud_credentials(dbt_cloud_account_id, dbt_cloud_project_id, dbt_cloud_api_token)

# print the credential metadata
dbt_cloud_credential_ids

[{'credential_id': 436894,
  'method': 'password',
  'username': 'steve_dowling_sa',
  'default_schema': 'dbt_sdowling_rs_exp'},
 {'credential_id': 436896,
  'method': 'password',
  'username': 'example_redshift_service_account',
  'default_schema': 'redshift_key_switch'}]

___Get secret from AWS secrets manager and update dbt Cloud Service account to use it for connection___

In [4]:
redshift_secret_name = "redshift-sales-cluster-service-account"
redshift_secret_region_name = "us-east-2"

# get redshift connection info from AWS secrets manager
aws_secrets_manager_redshift_conn_info = get_secret(my_secret_name, my_region_name)

# set username from what we got from AWS secrets manager
redshift_username = aws_secrets_manager_redshift_conn_info['username']

# set password from what we got from AWS secrets manager
redshift_password = aws_secrets_manager_redshift_conn_info['password']


# set the dbt Cloud Production environment Redshift credential ID for the project 390745 (Pulled from last command)
# I could do this programmatically, but for the sake of the example, I'm hardcoding it
dbt_cloud_credential_id = 436896

# update the dbt Cloud Production environment Redshift credentials for the project 390745
updating_creds = update_dbt_cloud_database_connection_credentials(
            account_id= dbt_cloud_account_id,
            project_id=dbt_cloud_project_id,
            credential_id=dbt_cloud_credential_id,
            dbt_cloud_token=dbt_cloud_api_token,
            username=redshift_username,
            password=redshift_password
        )

print("Successfully updated dbt cloud credential id 436896 with redshift username and password" if updating_creds['status']['code'] == 200 else "failed to update dbt cloud credential id 436896 with redshift username and password")

Successfully updated dbt cloud credential id 436896 with redshift username and password


# END OF SCRIPT